# Darts regression model

In [ ]:
import warnings

import pandas as pd
from darts.models import RegressionModel
from darts.dataprocessing.transformers import Scaler
from sklearn.compose import TransformedTargetRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from src.utils import prepare_col, historical_forecasts, plot_forecasts

## Data preparation

In [ ]:
dset = pd.read_csv("../results/dataset_minutes.csv")
dset

In [ ]:
train_materials = ["G5", "G6", "G8", "G9"]
test_materials = ["G7"]

In [ ]:
dset_train = dset[dset.material.isin(train_materials)]
groups_train = dset_train.groupby(["material", "sample"])
series_train = [prepare_col(group, "Wear Loss [mm]") for _, group in groups_train]
series_train_idx = [prepare_col(group, "Time [min]") for _, group in groups_train]

scaler = Scaler().fit([series_train[0], series_train_idx[0]])
series_train_scaled, series_train_idx_scaled = zip(
    *[scaler.transform(series) for series in zip(series_train, series_train_idx)]
)

## Model training

In [ ]:
mlp = make_pipeline(
    StandardScaler(),
    TransformedTargetRegressor(MLPRegressor(), transformer=StandardScaler()),
)

model = RegressionModel(
    lags=100, lags_future_covariates=[1], output_chunk_length=300, model=mlp
)

In [ ]:
%%time
with warnings.catch_warnings():
    warnings.filterwarnings("ignore")
    model.fit(series_train_scaled, future_covariates=series_train_idx_scaled)

## Predictions

In [ ]:
n_warmup = [1000, 2000]

In [ ]:
%%time
dset_pred = dset[dset.material.isin(train_materials + test_materials)]
forecasts = {
    key: historical_forecasts(model, scaler, group, n_warmup=n_warmup, n_lags=1)
    for key, group in dset_pred.groupby(["material", "sample"])
}

In [ ]:
_ = plot_forecasts(forecasts, n_warmup, n_col=4)